In [33]:
import json
import random
import yaml
from itertools import product

In [22]:
config_path = '../protosp03/config/synthetic.yaml'

In [31]:
config = yaml.load(open(config_path, "r"), Loader=yaml.FullLoader)

In [32]:
config['levels']

[{'L1': [{'SL1': [{'SSL1': 3}, {'SSL2': 2}]}, {'SL2': [{'SSL1': 5}]}]},
 {'L2': [{'SL1': [{'SSL1': 1}, {'SSL2': 4}]}]}]

In [43]:
levels = [3,3,5]
levels = [range(1, elem+1) for elem in levels]
for l in product(*levels):
    print(*l)

1 1 1
1 1 2
1 1 3
1 1 4
1 1 5
1 2 1
1 2 2
1 2 3
1 2 4
1 2 5
1 3 1
1 3 2
1 3 3
1 3 4
1 3 5
2 1 1
2 1 2
2 1 3
2 1 4
2 1 5
2 2 1
2 2 2
2 2 3
2 2 4
2 2 5
2 3 1
2 3 2
2 3 3
2 3 4
2 3 5
3 1 1
3 1 2
3 1 3
3 1 4
3 1 5
3 2 1
3 2 2
3 2 3
3 2 4
3 2 5
3 3 1
3 3 2
3 3 3
3 3 4
3 3 5


In [39]:
from itertools import product

# Define two ranges
range1 = range(1,3)  # 0, 1, 2
range2 = range(1,2)  # 0, 1

# Iterate over the cartesian product of the two ranges
for val1, val2 in product(range1, range2):
    print(val1, val2)


1 1
2 1


In [37]:
levels

[range(1, 4), range(1, 4), range(1, 6)]

In [5]:
lo_path = '../data/raw/evrlearn/job_profiles.json'
lo = json.load(open(lo_path, 'r'))

In [15]:
random.randint(0, 14)

0

In [9]:
lo[2]

{'id': 3,
 'name': 'Database Engineer',
 'level_id': 3,
 'level': 'Senior',
 'job_profile_skills': [{'id': 28,
   'job_profile_id': 3,
   'job_skill_id': 1,
   'level_id': 2,
   'created_at': '2023-07-25T09:51:24.705+02:00',
   'updated_at': '2023-07-25T09:51:24.705+02:00',
   'certificate': False},
  {'id': 29,
   'job_profile_id': 3,
   'job_skill_id': 2,
   'level_id': 3,
   'created_at': '2023-07-25T09:51:24.707+02:00',
   'updated_at': '2023-07-25T09:51:24.707+02:00',
   'certificate': False},
  {'id': 30,
   'job_profile_id': 3,
   'job_skill_id': 3,
   'level_id': 2,
   'created_at': '2023-07-25T09:51:24.716+02:00',
   'updated_at': '2023-07-25T09:51:24.716+02:00',
   'certificate': False},
  {'id': 31,
   'job_profile_id': 3,
   'job_skill_id': 4,
   'level_id': 3,
   'created_at': '2023-07-25T09:51:24.721+02:00',
   'updated_at': '2023-07-25T09:51:24.721+02:00',
   'certificate': False},
  {'id': 32,
   'job_profile_id': 3,
   'job_skill_id': 5,
   'level_id': 3,
   'created_a